In [ ]:
import random
import logging
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, Model
from sklearn.metrics import f1_score, recall_score, accuracy_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import pickle
import os
import librosa
from sklearn.model_selection import train_test_split
from transformers import AutoFeatureExtractor, Wav2Vec2FeatureExtractor, TFWav2Vec2Model, Wav2Vec2Processor

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)
# Set random seed
tf.keras.utils.set_random_seed(42)

In [ ]:
#!/bin/bash
!curl -L -o ./deep-voice-deepfake-voice-recognition.zip\
  https://www.kaggle.com/api/v1/datasets/download/birdy654/deep-voice-deepfake-voice-recognition

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3773M  100 3773M    0     0  38.7M      0  0:01:37  0:01:37 --:--:-- 39.7M


In [ ]:
# prompt: unzip the file

!unzip ./deep-voice-deepfake-voice-recognition.zip


Archive:  ./deep-voice-deepfake-voice-recognition.zip
  inflating: DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3  
  inflating: DEMONSTRATION/DEMONSTRATION/linus-to-musk-DEMO.mp3  
  inflating: KAGGLE/AUDIO/FAKE/Obama-to-Biden.wav  
  inflating: KAGGLE/AUDIO/FAKE/Obama-to-Trump.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-Obama.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-Trump.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-linus.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-margot.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-musk.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-ryan.wav  
  inflating: KAGGLE/AUDIO/FAKE/biden-to-taylor.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-biden.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-margot.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-musk.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-obama.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-ryan.wav  
  inflating: KAGGLE/AUDIO/FAKE/linus-to-taylor.wav  
  inflating: KAGGLE/AUDIO/FAKE/li

### Basic configuration

In [ ]:
# Maximum duration of the input audio file we feed to our Wav2Vec 2.0 model.
MAX_DURATION = 5  # Change to 30 seconds

# Sampling rate is the number of samples of audio recorded every second
SAMPLING_RATE = 16000
BATCH_SIZE = 8  # Batch-size for training and evaluating our model.
NUM_CLASSES = 2  # Number of classes our dataset will have (2 in our case).
HIDDEN_DIM = 768  # Dimension of our model output (768 in case of Wav2Vec 2.0 - Base).
MAX_SEQ_LENGTH = MAX_DURATION * SAMPLING_RATE  # Maximum length of the input audio file.

# Wav2Vec 2.0 results in an output frequency with a stride of about 20ms.
MAX_FRAMES = 249 # Adjust for 30 seconds
MAX_EPOCHS = 5  # Maximum number of training epochs.
SEED = 42
MODEL_CHECKPOINT = "facebook/wav2vec2-base"  # Name of pretrained model from Hugging Face Model Hu

FAKE = "KAGGLE/AUDIO/FAKE"
REAL = "KAGGLE/AUDIO/REAL"

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")  #retrieve feature extractor

### Load files

In [ ]:
def load_audio_files_and_labels(fake_folder, real_folder):
    audio_data = []
    labels = []

    # Load fake audio files
    for filename in os.listdir(fake_folder):
        file_path = os.path.join(fake_folder, filename)
        audio, sr = librosa.load(file_path, sr=16000, duration=MAX_DURATION)  # Load 30 seconds
        audio_data.append(audio)
        labels.append(1)

    # Load real audio files
    for filename in os.listdir(real_folder):
        file_path = os.path.join(real_folder, filename)
        audio, sr = librosa.load(file_path, sr=16000, duration=MAX_DURATION)  # Load 30 seconds
        audio_data.append(audio)
        labels.append(0)

    return audio_data, np.array(labels)

def extract_features(audio_data):
    features = []
    for audio in audio_data:
        inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="np", padding=True, truncation=True, max_length=MAX_SEQ_LENGTH)
        features.append(inputs.input_values)
    return np.concatenate(features, axis=0)

In [ ]:
audio_data, labels = load_audio_files_and_labels(FAKE, REAL)
features = extract_features(audio_data)
print(features.shape)

(64, 80000)


### Create training and testing batch

In [ ]:
X_tr, X_te, y_tr, y_te = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=SEED)

### Create custom Model using TFWav2Vec2

In [ ]:

def mean_pool(hidden_states, feature_lengths, batch_size):
    attenion_mask = tf.sequence_mask(
        feature_lengths, maxlen=MAX_FRAMES, dtype=tf.dtypes.int64
    )
    padding_mask = tf.cast(
        tf.reverse(tf.cumsum(tf.reverse(attenion_mask, [-1]), -1), [-1]),
        dtype=tf.dtypes.bool,
    )
    hidden_states = tf.where(
        tf.broadcast_to(
            tf.expand_dims(~padding_mask, -1), (batch_size, MAX_FRAMES, HIDDEN_DIM)
        ),
        0.0,
        hidden_states,
    )
    pooled_state = tf.math.reduce_sum(hidden_states, axis=1) / tf.reshape(
        tf.math.reduce_sum(tf.cast(padding_mask, dtype=tf.dtypes.float32), axis=1),
        [-1, 1],
    )
    return pooled_state

class Wav2Vec2_Model(layers.Layer):

    def __init__(self, model_checkpoint, num_classes):
        super().__init__()
        self.wav2vec2 = TFWav2Vec2Model.from_pretrained(
            model_checkpoint, apply_spec_augment=False, from_pt=True
        )
        self.pooling = layers.GlobalAveragePooling1D()
        self.intermediate_layer_dropout = layers.Dropout(0.5)
        self.final_layer = layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        hidden_states = self.wav2vec2(inputs["input_values"])[0]
        batch_size = tf.shape(hidden_states)[0]

        if tf.is_tensor(inputs["attention_mask"]):
            audio_lengths = tf.cumsum(inputs["attention_mask"], -1)[:, -1]
            feature_lengths = self.wav2vec2.wav2vec2._get_feat_extract_output_lengths(
                audio_lengths
            )
            pooled_state = mean_pool(hidden_states, feature_lengths, batch_size)
        else:
            pooled_state = self.pooling(hidden_states)

        intermediate_state = self.intermediate_layer_dropout(pooled_state)
        final_state = self.final_layer(intermediate_state)

        return final_state

# Rebuild the model to apply the mixed precision policy
def build_model():
    inputs = {
        "input_values": tf.keras.Input(shape=(MAX_SEQ_LENGTH,), dtype="float32"),
        "attention_mask": tf.keras.Input(shape=(MAX_SEQ_LENGTH,), dtype="int32"),
    }
    wav2vec2_model = Wav2Vec2_Model(MODEL_CHECKPOINT, NUM_CLASSES)(
        inputs
    )
    model = tf.keras.Model(inputs, wav2vec2_model)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    return model

model = build_model()

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(

TFWav2Vec2Model has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFWav2Vec2Model: ['quantizer.weight_proj.bias', 'project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.weight', 'project_q.bias']
- This IS expected if you are initializing TFWav2Vec2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a Bert

In [ ]:
inputs = tf.data.Dataset.from_tensor_slices(({"input_values": X_tr, "attention_mask": np.ones_like(X_tr)}, y_tr)).batch(BATCH_SIZE)
val_data = tf.data.Dataset.from_tensor_slices(({"input_values": X_te, "attention_mask": np.ones_like(X_te)}, y_te)).batch(BATCH_SIZE)

# Training
history = model.fit(
    inputs,
    validation_data=val_data,
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS
)

Epoch 1/5
7/7 [==============================] - 59s 2s/step - loss: 0.3908 - accuracy: 0.8627 - val_loss: 0.4153 - val_accuracy: 0.8462
Epoch 2/5
7/7 [==============================] - 5s 700ms/step - loss: 0.3308 - accuracy: 0.8824 - val_loss: 0.3781 - val_accuracy: 0.8462
Epoch 3/5
7/7 [==============================] - 5s 708ms/step - loss: 0.2875 - accuracy: 0.8824 - val_loss: 0.3268 - val_accuracy: 0.8462
Epoch 4/5
7/7 [==============================] - 5s 704ms/step - loss: 0.2571 - accuracy: 0.8824 - val_loss: 0.2728 - val_accuracy: 0.8462
Epoch 5/5
7/7 [==============================] - 5s 719ms/step - loss: 0.2150 - accuracy: 0.9216 - val_loss: 0.2181 - val_accuracy: 0.8462


### Measure model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Get training accuracy list
training_accuracy = history.history['accuracy']
print("Training Accuracy List: ", training_accuracy)


# Get predictions
y_pred_probs = model.predict(
    {"input_values": X_te, "attention_mask": np.ones_like(X_te)},
    batch_size=BATCH_SIZE
)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_te, y_pred)

# Calculate precision, recall, and F1-score
precision = precision_score(y_te, y_pred, average='binary')
recall = recall_score(y_te, y_pred, average='binary')
f1 = f1_score(y_te, y_pred, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Training Accuracy List:  [0.8627451062202454, 0.8823529481887817, 0.8823529481887817, 0.8823529481887817, 0.9215686321258545]
2/2 [==============================] - 5s 162ms/step
Accuracy: 0.8461538461538461
Precision: 0.8461538461538461
Recall: 1.0
F1-score: 0.9166666666666666
